**Learning outcomes**

* Implement a linear combination of unitaries.

In [67]:
import numpy as np
import pennylane as qml

Trotterization approximates time evolution by reexpressing the full Taylor series as a product of simpler Taylor series. Another way to approximate this evolution is to truncate the Taylor series at some term $K$:
$$e^{-it\hat{H}/\hbar} \approx 1 + (\frac{it\hat{H}}{\hbar}) + ... + \frac{1}{k!}(\frac{it\hat{H}}{\hbar})^K$$
If $K$ is large, this should be a good approsimation. if $\hat{H}$ is a weighted sum of unitary operators (as in the case of spin, where terms are built of out Pauli operators, e.g., $\alpha Z\otimes Z$), then each term in the sum will be proportional to a unitary, since a product of unitary matrices is unitary. So the problem reduces to impelemnting a **linear combination of unitaries (LCU)**

As a warm-up, let's consider the problem of adding two unitaries, $U$ and $V$ and applying them to an arbitrary state:
$$\Ket{\psi} \rightarrow (U + V)\Ket{\psi}$$

The sum $U+V$ won't be unitary in general, so there is no gate which applies it with certainty. However, we can *non-deterministically* apply it, in the sense that we can write down a circuit which applies it with some probability less than one. Here is the circuit:
![circuit](./images/H.6.1.1.png)

The top wire is an auxiliary qubit we measure as part of our circuit, while the bottom wire is the main register carrying the state we want to apply the combined operation to. If the outcome is $0$, then the outcome (up to normalization) is
$$(U + V)\Ket{\psi}$$
Let's code this up and check it works for a concrete example.



**Codercise H.6.1.**
(a) Write a circuit for applying a sum of unitaries non-deterministically. Don't worry about initialization of the state just yet.

Tip. Use [qml.ControlledQubitUnitary](https://docs.pennylane.ai/en/stable/code/api/pennylane.ControlledQubitUnitary.html) to apply these unitaries with control bits.

In [68]:
aux = 0
main = 1
n_bits = 2
dev = qml.device("default.qubit", wires=n_bits)

def add_two_unitaries(U, V):
    """A circuit to apply the sum of two unitaries non-deterministically.

    Args:
        U (array): A unitary matrix, stored as a complex array.
        V (array): A unitary matrix, stored as a complex array.
    """
    qml.Hadamard(wires=aux)

    qml.ControlledQubitUnitary(U,control_wires=[0], wires=1, control_values="0")
    qml.ControlledQubitUnitary(V, control_wires=[0], wires=1, control_values="1")

    qml.Hadamard(wires=aux)

(b) Complete the code below to apply the sum $X + Z$ to the state $\Ket{0}$ (on the main register). You can invoke `add_two_unitaries(U, V)` from the last challenge, and access the matrix form of the Paulis using, e.g., `qml.PauliX.matrix`.


In [69]:
@qml.qnode(dev)
def X_plus_Z():
    """Apply X + Z to |0> and return the state."""
    U = qml.PauliX.compute_matrix()
    V = qml.PauliZ.compute_matrix()
    add_two_unitaries(U,V)
    return qml.state()

print("The amplitudes on the main register are proportional to", X_plus_Z()[:2], ".")
print(qml.draw(X_plus_Z)())

The amplitudes on the main register are proportional to [0.5+0.j 0.5+0.j] .
0: ──H─╭○─────╭●──────H─┤  State
1: ────╰U(M0)─╰U(M1)────┤  State


IT turns out that this circuit generalizes to a sum of unitaries
$$\widetilde{U} = U_0 + U_1 + ... + U_{K-1}$$
provided the number of unitaries happens to be a binary power, $K = 2^k$. In that case, we can associate a binary control string to each component unitary $U_j$, which is just the expansion of $j$ in binary. Here is the circuit:
![circuit](./images/H.6.2.1.png)
The section in yhe middle is called the *multiplexer* or **SELECT subroutine**. The Hadamards acting on the auxiliary are called the **PREPARE subroutine**.

**Codercise H.6.2.**
(a) Finish the code below to implement the SELECT subcircuit.

In [70]:
k_bits = 2
n_bits = 2
all_bits = k_bits + n_bits
aux = range(k_bits)
main = range(k_bits, all_bits)
dev = qml.device("default.qubit", wires=all_bits)

def SELECT_uniform(U_list):
    """Implement the SELECT subroutine for 2^k unitaries.

    Args:
        U_list (list[array[complex]]): A list of unitary matrices, stored as
        complex arrays.
    """
    for index in range(2**k_bits):
        ctrl_str =  np.binary_repr(index, k_bits) # Create binary representation
        qml.ControlledQubitUnitary(U_list[index], control_wires=[ _ for _ in aux], wires=[_ for _ in main], control_values=ctrl_str)

(b) Write a circuit to apply
$$X \otimes H + H \otimes Z = X \otimes X + Z \otimes Z + X \otimes Z + Z \otimes X = U_0 + U_1 + U_2 + U_3$$
to the state $\Ket{01}$, state on the `main` register. You can access the auxiliary wires as `aux` and the `SELECT_uniform` function from the previous exercise is available. Note that
$$U_0 + U_1 + U_2 + U_3  \varpropto H \otimes H$$

In [71]:
@qml.qnode(dev)
def XH_plus_HZ():
    """Apply XH + HZ to |01> and return the state."""
    U_list = [np.kron(qml.PauliX.compute_matrix(), qml.PauliX.compute_matrix()),
              np.kron(qml.PauliZ.compute_matrix(), qml.PauliZ.compute_matrix()),
              np.kron(qml.PauliX.compute_matrix(), qml.PauliZ.compute_matrix()),
              np.kron(qml.PauliZ.compute_matrix(), qml.PauliX.compute_matrix())]
    # state preparation |01>
    qml.PauliX(wires= main[-1])

    #PREPARE subroutine
    qml.broadcast(qml.Hadamard, wires=[i for i in aux], pattern="single")

    #multiplexer
    SELECT_uniform(U_list)

    qml.broadcast(qml.Hadamard, wires=[i for i in aux], pattern="single")
    return qml.state()

print("The amplitudes on the main register are proportional to", XH_plus_HZ()[:4], ".")

print(qml.draw(XH_plus_HZ)())

The amplitudes on the main register are proportional to [ 0.25+0.j -0.25+0.j  0.25+0.j -0.25+0.j] .
0: ──H─╭○─────╭○─────╭●─────╭●──────H─┤  State
1: ──H─├○─────├●─────├○─────├●──────H─┤  State
2: ────├U(M0)─├U(M1)─├U(M2)─├U(M3)────┤  State
3: ──X─╰U(M0)─╰U(M1)─╰U(M2)─╰U(M3)────┤  State


This state is proportional tp $\Ket{+} \otimes \Ket{-}= (H \otimes H)\Ket{01}$

So far, we have only dealt with equally weighted unitaries. To perform an unequally weighted linea combination,
$$\tilde{U} = \kappa U + V,$$
we require the sightly more involved circuit:
![circuit](./images/H.6.3.1.png)
where we have replaced The Hadamard gate with a more complicated gate:
$$V_{\kappa} = \frac{1}{\sqrt{\kappa +1}}\begin{bmatrix}
\sqrt{\kappa} & -1 \\
1 & \sqrt{\kappa}
\end{bmatrix}$$

Once again, we apply the weighted sum if we observe $0$ on the auxiliary qubit. It turns out that this two-unitary circuit is all you need to implement a general linear combination of unitaries, since you can iteratively nest this circuit to perform a weighted sum of multiple unitaries. We can finally revisit our original goal of approximating a Taylor series. The case for exponentiating a Hamiltonian which is itself a linear combination of unitaries is very similar, just more complicated, so we'll just focun on $e^{tU}$.

**Codercise H.6.3.** (a) COnsider the matrix exponential of a unitary operator $U$:
$$e^{tU} = I +tU+\frac{1}{2}t^2U^2 + ...$$
Code up the circuit to non-deterministically implement the first-order approximation of this Taylor series,
$$I + tU = U_0 + tU_1$$
this is fiven by the circuit:
![circuit](./images/H.6.3.2.png)
The auxiliary qubit is on wire `aux = 0` and the main qubit on `main=1`. Ass with `add_two_unitaries`, don't initialize the sate just yet.

In [85]:
def V(t):
    """Matrix for the PREPARE subroutine."""
    return np.array([[np.sqrt(t)/np.sqrt(t+1), -1/np.sqrt(t+1)],
                     [1/np.sqrt(t+1), np.sqrt(t)/np.sqrt(t+1)]])

def exp_U_first(U, t):
    """Implement the first two terms in the Taylor series for exp(tU).

    Args:
        U (array): A unitary matrix, stored as a complex array.
        t (float): A time to evolve by.
    """

    qml.QubitUnitary(V(t), wires=0)

    qml.ControlledQubitUnitary(U,control_wires=0, wires=1 ,control_values="0")

    qml.adjoint(qml.QubitUnitary)(V(t),wires=0)


(b) Now implement the second-order approximation to $e^{tU}$:
$$I+tU+\frac{1}{2}t^2U^2 = U_0 + t(U^1 + \frac{1}{2}tU^2)$$
Now `aux = [0,1]` and `main =2`. the trick here is to create a circuit corresponding to the term inn brackets:
![circuit](./images/H.6.3.3.png)
In other words, we have a *controlled subcircuit* realizing this second term.
Tip. You can create controlled subcircuits using [qml.ctrl](https://docs.pennylane.ai/en/stable/code/api/pennylane.ctrl.html). Make sure to properly specify the control values!

In [96]:
def exp_U_second(U, t):
    """Implement the second-order approximation of exp(tU).

    Args:
        U (array): A unitary matrix, stored as a complex array.
        t (float): A time to evolve by.
    """
    qml.QubitUnitary(V(t), wires=aux[0])

    def subcircuit():
        qml.QubitUnitary(V(t/2), wires=aux[1])
        qml.ControlledQubitUnitary(np.matmul(U,U),control_wires=aux[1], wires=main ,control_values="0")
        qml.ControlledQubitUnitary(U,control_wires=aux[1], wires=main ,control_values="1")
        qml.adjoint(qml.QubitUnitary)(V(t/2),wires=aux[1])

    # ADD CONTROLLED OPERATION HERE
    qml.PauliX(wires=aux[0])
    ctr = qml.ctrl(subcircuit,aux[0])
    ctr()
    qml.PauliX(wires=aux[0])
    qml.adjoint(qml.QubitUnitary)(V(t),wires=aux[0])

(c) Finally, write circuits which implement the evolution $e^{itX}$, starting in the $\Ket{0}$ state, using the (i) the first-order approximation, (ii) the second-order approximation, and (iii) the full series via `qml.RX`. The functions `exp_U_first` and `exp_U_second` from the previous two exercises are available.
Hitting the submit button will plot the normalized coefficient of  for the main register qubit. The first-order results are red, second-order blue and full series is green. Do the results make sense?

In [98]:
aux = [0, 1]
main = 2
all_bits = range(3)
dev = qml.device("default.qubit", wires=all_bits)

# Part (i)

@qml.qnode(dev)
def first_approx(t):
    exp_U_first(1j*qml.PauliX.compute_matrix(),t)
    return qml.state()

# Part (ii)

@qml.qnode(dev)
def second_approx(t):
    exp_U_second(1j*qml.PauliX.compute_matrix(),t)
    return qml.state()

# Part (iii)

@qml.qnode(dev)
def full_series(t):
    qml.RX(-2*t, wires=main)
    return qml.state()

##################
# HIT SUBMIT FOR #
# PLOTTING MAGIC #
##################

![plot](./images/H.6.3.4.png)